In [1]:
from tensorflow.python.client import device_lib
# 列出所有的本地机器设备
local_device_protos = device_lib.list_local_devices()
# 只打印GPU设备
[print(x) for x in local_device_protos if x.device_type == 'GPU']

name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7470045594
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2322274737463933412
physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1"



[None]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
print(np.random.randint(1, 6, size=10))

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.python.client import device_lib
# 列出所有的本地机器设备
local_device_protos = device_lib.list_local_devices()
# 只打印GPU设备
[print(x) for x in local_device_protos if x.device_type == 'GPU']


n_epochs = 100
n_episodes = 100
n_way = 20
n_shot = 3
n_query = 15
n_examples = 350
im_width, im_height, channels = 84, 84, 3
h_dim = 64
z_dim = 64

# Load Train Dataset
train_dataset = np.load('/content/drive/My Drive/data/mini-imagenet-train_600.npy')
train_dataset = train_dataset[:,:n_examples]
n_classes = train_dataset.shape[0]
print(train_dataset.shape)

test_dataset = np.load('/content/drive/My Drive/data/mini-imagenet-test_600.npy')
n_test_classes = test_dataset.shape[0]
print(test_dataset.shape)
n_test_episodes = 600
n_test_examples = 600
n_test_way = 20
n_test_shot = 3
n_test_query = 15

'''
epi_classes = np.random.permutation(n_classes)[:n_way]
support = np.zeros([n_way, n_shot, im_height, im_width, channels], dtype=np.float32)
query = np.zeros([n_way, n_query, im_height, im_width, channels], dtype=np.float32)
for i, epi_cls in enumerate(epi_classes):
    selected = np.random.permutation(n_examples)[:n_shot + n_query]
    support[i] = train_dataset[epi_cls, selected[:n_shot]]
    query[i] = train_dataset[epi_cls, selected[n_shot:]]
# support = np.expand_dims(support, axis=-1)
# query = np.expand_dims(query, axis=-1)
labels = np.tile(np.arange(n_way)[:, np.newaxis], (1, n_query)).astype(np.uint8)
'''


def conv_block(inputs, out_channels, name='conv'):
    with tf.variable_scope(name):
        conv = tf.layers.conv2d(inputs, out_channels, kernel_size=3, padding='SAME')
        conv = tf.contrib.layers.batch_norm(conv, updates_collections=None, decay=0.99, scale=True, center=True)
        conv = tf.nn.relu(conv)
        conv = tf.contrib.layers.max_pool2d(conv, 2)
        return conv


def encoder(x, h_dim, z_dim, reuse=False):
    with tf.variable_scope('encoder', reuse=reuse):
        net = conv_block(x, h_dim, name='conv_1')
        net = conv_block(net, h_dim, name='conv_2')
        net = conv_block(net, h_dim, name='conv_3')
        net = conv_block(net, z_dim, name='conv_4')
        net = tf.contrib.layers.flatten(net)
        return net


def euclidean_distance(a, b):
    # a.shape = N x D
    # b.shape = M x D
    N, D = tf.shape(a)[0], tf.shape(a)[1]
    M = tf.shape(b)[0]
    a = tf.tile(tf.expand_dims(a, axis=1), (1, M, 1))
    b = tf.tile(tf.expand_dims(b, axis=0), (N, 1, 1))
    return tf.reduce_mean(tf.square(a - b), axis=2)


name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7470045594
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7026712244002112254
physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1"

(64, 350, 84, 84, 3)
(20, 600, 84, 84, 3)


In [0]:
def oversampling(batch_data, few_shot):
  balanced_data = np.zeros(batch_data.shape)
  balanced_data = np.copy(batch_data)
  temp_shot = len(batch_data[0])
  for i, n_few in enumerate(few_shot):
    choosed = np.random.randint(0, n_few, size=temp_shot - n_few)
    balanced_data[i, n_few:] = batch_data[i, choosed]
  return balanced_data


def require_unbalanced_data(train_dataset,
               n_classes=n_classes,
               n_way=n_way,
               n_shot=n_shot,
               n_query=n_query,
               n_examples=n_examples):
  epi_classes = np.random.permutation(n_classes)[:n_way]
  n_few_shot = np.random.randint(1, n_shot+1, size=n_way)
  support = np.zeros([n_way, n_shot, im_height, im_width, channels], dtype=np.float32)
  query = np.zeros([n_way, n_query, im_height, im_width, channels], dtype=np.float32)
  for i, epi_cls in enumerate(epi_classes):
    selected = np.random.permutation(n_examples)[:n_few_shot[i] + n_query]
    support[i,:n_few_shot[i]] = train_dataset[epi_cls, selected[:n_few_shot[i]]]
    query[i] = train_dataset[epi_cls, selected[n_few_shot[i]:]]
  labels = np.tile(np.arange(n_way)[:, np.newaxis], (1, n_query)).astype(np.uint8)
  return support, query, labels, n_few_shot


def uniform_sampling(train_dataset,
           n_classes=n_classes,
           n_way=n_way,
           n_shot=n_shot,
           n_query=n_query,
           n_examples=n_examples):
  epi_classes = np.random.permutation(n_classes)[:n_way]
  support = np.zeros([n_way, n_shot, im_height, im_width, channels], dtype=np.float32)
  query = np.zeros([n_way, n_query, im_height, im_width, channels], dtype=np.float32)
  for i, epi_cls in enumerate(epi_classes):
      selected = np.random.permutation(n_examples)[:n_shot + n_query]
      support[i] = train_dataset[epi_cls, selected[:n_shot]]
      query[i] = train_dataset[epi_cls, selected[n_shot:]]
  labels = np.tile(np.arange(n_way)[:, np.newaxis], (1, n_query)).astype(np.uint8)
  return support, query, labels


def test_bench(sess,
        test_dataset,
        n_test_classes = n_test_classes,
        n_test_episodes=n_test_episodes, 
        n_test_way = n_test_way,
        n_test_shot = n_test_shot,
        n_test_query = n_test_query,
        n_test_examples=n_test_examples):
  print('Testing...')
  avg_acc = 0.
  n_test_classes = test_dataset.shape[0]
  for epi in range(n_test_episodes):
    support, query, labels = uniform_sampling(test_dataset,
           n_classes=n_test_classes,
           n_way=n_test_way,
           n_shot=n_test_shot,
           n_query=n_test_query,
           n_examples=n_test_examples)
    # support, query, labels, n_few_shot = require_unbalanced_data(test_dataset,
    #                                 n_classes=n_test_classes,
    #                                 n_way=n_test_way,
    #                                 n_shot=n_test_shot,
    #                                 n_query=n_test_query,
    #                                 n_examples=n_test_examples)
    # support = oversampling(support, n_few_shot)
    ls, ac, = sess.run([ce_loss, acc], feed_dict={x: support, q: query, y:labels})
    avg_acc += ac
  avg_acc /= n_test_episodes
  print('Average Test Accuracy: {:.5f}'.format(avg_acc))
  return None

In [5]:
x = tf.placeholder(tf.float32, [None, None, im_height, im_width, channels])
q = tf.placeholder(tf.float32, [None, None, im_height, im_width, channels])
var_weight = tf.get_variable(name="variance_weight", shape=(1, 1))
var_b = tf.get_variable(name="variance_bias", shape=(1))
x_shape = tf.shape(x)
q_shape = tf.shape(q)
num_classes, num_support = x_shape[0], x_shape[1]
num_queries = q_shape[1]
y = tf.placeholder(tf.int64, [None, None])
y_one_hot = tf.one_hot(y, depth=num_classes)
emb_in = encoder(tf.reshape(x, [num_classes * num_support, im_height, im_width, channels]), h_dim, z_dim)
emb_dim = tf.shape(emb_in)[-1]
emb_x, q_variance = tf.nn.moments(tf.reshape(emb_in, [num_classes, num_support, emb_dim]), axes=1)
m_variance = tf.reshape(tf.reduce_mean(q_variance, axis=1), [num_classes, 1])
emb_variance = tf.nn.bias_add(tf.matmul(m_variance, var_weight), var_b)
emb_variance = tf.reshape(tf.nn.softmax(emb_variance, axis=0), [1, num_classes])

# emb_variance = tf.nn.softmax(emb_variance, axis=0)
emb_var = tf.tile(emb_variance, [num_queries * num_classes, 1])
# emb_x = tf.reduce_mean(tf.reshape(emb_in, [num_classes, num_support, emb_dim]), axis=1)
emb_q = encoder(tf.reshape(q, [num_classes * num_queries, im_height, im_width, channels]),
                h_dim, z_dim, reuse=True)

# q_mean, q_variance = tf.nn.moments(emb_q, axes=1)
# emb_q = (emb_q - q_mean) / q_variance
dists = euclidean_distance(emb_q, emb_x)
# amend_dists = tf.multiply(emb_var, dists) * tf.to_float(num_classes)
log_p_y = tf.reshape(tf.nn.log_softmax(-dists), [num_classes, num_queries, -1])
ce_loss = -tf.reduce_mean(tf.reshape(tf.reduce_sum(tf.multiply(y_one_hot, log_p_y), axis=-1), [-1]))
acc = tf.reduce_mean(tf.to_float(tf.equal(tf.argmax(log_p_y, axis=-1), y)))

train_op = tf.train.AdamOptimizer().minimize(ce_loss)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)
init_op = tf.global_variables_initializer()
sess.run(init_op)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use `tf.cast` instead.


In [6]:
n_epochs = 100
for ep in range(n_epochs):
    for epi in range(n_episodes):
        support, query, labels = uniform_sampling(train_dataset)
        # support, query, labels, n_few_shot = require_unbalanced_data(train_dataset)
        # support = oversampling(support, n_few_shot)
        _, ls, ac = sess.run([train_op, ce_loss, acc], feed_dict={x: support, q: query, y:labels})
        if (epi+1) % 50 == 0:
            std_str = '[epoch {}/{}, episode {}/{}] => loss: {:.5f}, acc: {:.5f}'
            print(std_str.format(ep+1, n_epochs, epi+1, n_episodes, ls, ac))
        if (ep + 1) % 3 == 0 and epi == 0:
            # w, b, m_v, emb_v = sess.run([var_weight, var_b, m_variance, emb_variance], 
            #             feed_dict={x: support, q: query, y:labels})
            # print("w:", w, "b:", b)
            # print("mean_var:", m_v)
            # print("emb_v:", emb_v)
            test_bench(sess=sess,test_dataset=test_dataset)
print("Training is over.")
test_bench(sess=sess, test_dataset=test_dataset)

[epoch 1/100, episode 50/100] => loss: 2.77812, acc: 0.16333
[epoch 1/100, episode 100/100] => loss: 2.82944, acc: 0.13333
[epoch 2/100, episode 50/100] => loss: 2.52436, acc: 0.23000
[epoch 2/100, episode 100/100] => loss: 2.67269, acc: 0.20333
Testing...
Average Test Accuracy: 0.16866
[epoch 3/100, episode 50/100] => loss: 2.45765, acc: 0.28667
[epoch 3/100, episode 100/100] => loss: 2.63808, acc: 0.19000
[epoch 4/100, episode 50/100] => loss: 2.61280, acc: 0.18333
[epoch 4/100, episode 100/100] => loss: 2.49701, acc: 0.25000
[epoch 5/100, episode 50/100] => loss: 2.36323, acc: 0.27000
[epoch 5/100, episode 100/100] => loss: 2.35664, acc: 0.30333
Testing...
Average Test Accuracy: 0.19887
[epoch 6/100, episode 50/100] => loss: 2.57100, acc: 0.21667
[epoch 6/100, episode 100/100] => loss: 2.35643, acc: 0.29333
[epoch 7/100, episode 50/100] => loss: 2.18879, acc: 0.33000
[epoch 7/100, episode 100/100] => loss: 2.32065, acc: 0.31000
[epoch 8/100, episode 50/100] => loss: 2.08130, acc: 0.

In [0]:
'''
import glob
import os
import numpy as np
import cv2

n_train_classes = 64
n_test_classes = 20
n_examples, width, height, channels = 600, 84, 84, 3

root_path = '/content/drive/My Drive/data/miniimagenet'
train_path = os.path.join(root_path, 'train')
test_path = os.path.join(root_path, 'test')

train_dirs = [f for f in glob.glob(os.path.join(train_path, '*')) if os.path.isdir(f)]
test_dirs = [f for f in glob.glob(os.path.join(test_path, '*')) if os.path.isdir(f)]
print(len(train_dirs))

assert len(train_dirs) == n_train_classes
assert len(test_dirs) == n_test_classes

read_and_resize = lambda x: cv2.imread(x, 1)

def sample_dataset(dataset, dirs, name='train'):
    for i, d in enumerate(dirs):
        fs = np.asarray(glob.glob(os.path.join(d, '*.jpg')))
        fs = fs[np.random.permutation(len(fs))][:n_examples]
        for j, f in enumerate(fs):
            dataset[i, j] = read_and_resize(f)
        print('{}: {} of {}'.format(name, i + 1, len(dirs)))
    return dataset

train_dataset = np.zeros((n_train_classes, n_examples, width, height, channels), dtype=np.uint8)
train_dataset = sample_dataset(train_dataset, train_dirs)
np.save('/content/drive/My Drive/data/mini-imagenet-train_600.npy', train_dataset)
del train_dataset

test_dataset = np.zeros((n_test_classes, n_examples, width, height, channels), dtype=np.uint8)
test_dataset = sample_dataset(test_dataset, test_dirs, name='test')
np.save('/content/drive/My Drive/data/mini-imagenet-test_600.npy', test_dataset)
'''

In [0]:
img = tf.Variable(tf.random_normal([3, 4]))
axis = list(range(len(img.get_shape()) - 1))
mean, variance = tf.nn.moments(img, axis)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    rs = sess.run(img)
    print(rs)
    print(rs.shape)
    print(axis)
    resultMean = sess.run(mean)
    print('resultMean',resultMean)
    resultVar = sess.run(variance)
    print('resultVar',resultVar)


[[-0.15628773 -0.04902783  1.0848862  -1.4387043 ]
 [-1.7748989  -1.9277954   1.2477913  -0.14888158]
 [-0.31883678 -0.5837216  -2.0567434   0.48701447]]
(3, 4)
[0]
resultMean [-0.7500078  -0.8535149   0.09197799 -0.36685714]
resultVar [0.5296046  0.62468886 2.312925   0.6418221 ]


In [0]:
#!unzip miniimagenet.zip > /dev/null